# MIMIC-III Extubation Failure Analysis

### Code to Select Heart Rate Information from the CHARTEVENTS CSV in MIMIC Clinical

In this section, we will use the Polars library to process the large volume of data present in the CHARTEVENTS CSV file from the MIMIC-III dataset. This will allow us to efficiently extract and analyze ***heart rate*** information relevant to our study on extubation failure in ICUs.

In [9]:
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt

In [10]:
# Define the path to the CHARTEVENTS CSV file
CHARTEVENTS = '/Volumes/FelipeSSD/mimic_clinical/CHARTEVENTS.csv'

In [11]:
# Define the data types for specific columns
# Here, we specify that the "VALUE" column should be treated as a string (Utf8)
dtypes = {"VALUE": pl.Utf8}

# Define values that should be treated as null
# "Moves on Bed" is specified as a null value
null_values = ["Moves on Bed"]

# Load the CHARTEVENTS CSV file using Polars with specific settings
# infer_schema_length is set to 100000 to adjust the number of rows used to infer the schema
# dtypes specifies the data types for columns
# null_values specifies values that should be treated as null
charevents = pl.scan_csv(CHARTEVENTS, infer_schema_length=100000, dtypes=dtypes, null_values=null_values)

FileNotFoundError: No such file or directory (os error 2): /Volumes/FelipeSSD/mimic_clinical/CHARTEVENTS.csv

### Filtering the Heart Rate Information
During the search for the `ITEMID` related to the Heart Rate we found 211 and 220045. We opted to choose the one with the most the available. Therefore we opted to conduct the analysis with the 220045 ID

In [4]:
heart_rate = charevents.filter(pl.col("ITEMID") == 211)
heart_rate = heart_rate.collect()

In [4]:
heart_rate_2 = charevents.filter(pl.col("ITEMID") == 220045)
heart_rate_2 = heart_rate_2.collect()

In [6]:
selected_patients = pl.read_csv('/Users/felipeakiomatsuoka/Desktop/PIBIC/csvs/chartevents_processed.csv')
patients_ids = sorted(list(selected_patients['SUBJECT_ID'].unique()))

#heart_rate_extubation_1 = heart_rate.filter(pl.col("SUBJECT_ID").is_in(patients_ids))
heart_rate_extubation_2 = heart_rate_2.filter(pl.col("SUBJECT_ID").is_in(patients_ids))

In [ ]:
print(f"There are {len(heart_rate_extubation_1)} rows in the dataframe")
print(f"There are {len(heart_rate_extubation_1['SUBJECT_ID'].unique())} unique patients")
print(f"" )

print(f"There are {len(heart_rate_extubation_2)} rows in the dataframe")
print(f"There are {len(heart_rate_extubation_2['SUBJECT_ID'].unique())} unique patients")

There are 71009 rows in the dataframe
There are 318 unique patients
There are 1121759 rows in the dataframe
There are 3999 unique patients


### Match the Heart Rate with the Extubation Processed Dataset

This section of the code reads in a processed dataset of patients and merges it with heart rate data to align heart rate measurements with the extubation process. The goal is to analyze heart rate trends in relation to the extubation events.



In [8]:
processed_patients = pd.read_csv('/Users/felipeakiomatsuoka/Desktop/PIBIC/csvs/chartevents_processed.csv')

heart_rate_data = heart_rate_extubation_2[['SUBJECT_ID', 'ICUSTAY_ID', 'CHARTTIME', 'VALUENUM', 'VALUEUOM']]
heart_rate_data = pd.merge(processed_patients, heart_rate_data, on='ICUSTAY_ID')

NameError: name 'heart_rate_extubation_2' is not defined

In [ ]:
heart_rate_data.head()

,Unnamed: 0,ICUSTAY_ID,SUBJECT_ID_x,STARTTIME,ENDTIME,TIME_SPENT,PREVIOUS_PROCEDURE_END,TIME_SINCE_LAST_INTUBATION,START_ANALYSIS,END_ANALYSIS,LABEL,DEAD,DEATH_TIME,SUBJECT_ID_y,CHARTTIME,VALUENUM,VALUEUOM
0,3,200033.0,56369,2198-08-10 17:30:00,2198-08-20 12:20:00,234.833333,2198-08-10 17:30:00,0.0,2198-08-20 12:20:00,2198-08-21 12:20:00,0,1,2198-08-21 00:00:00,56369,2198-08-08 17:00:00,63.0,bpm
1,3,200033.0,56369,2198-08-10 17:30:00,2198-08-20 12:20:00,234.833333,2198-08-10 17:30:00,0.0,2198-08-20 12:20:00,2198-08-21 12:20:00,0,1,2198-08-21 00:00:00,56369,2198-08-08 18:00:00,59.0,bpm
2,3,200033.0,56369,2198-08-10 17:30:00,2198-08-20 12:20:00,234.833333,2198-08-10 17:30:00,0.0,2198-08-20 12:20:00,2198-08-21 12:20:00,0,1,2198-08-21 00:00:00,56369,2198-08-08 19:00:00,61.0,bpm
3,3,200033.0,56369,2198-08-10 17:30:00,2198-08-20 12:20:00,234.833333,2198-08-10 17:30:00,0.0,2198-08-20 12:20:00,2198-08-21 12:20:00,0,1,2198-08-21 00:00:00,56369,2198-08-08 20:00:00,57.0,bpm
4,3,200033.0,56369,2198-08-10 17:30:00,2198-08-20 12:20:00,234.833333,2198-08-10 17:30:00,0.0,2198-08-20 12:20:00,2198-08-21 12:20:00,0,1,2198-08-21 00:00:00,56369,2198-08-08 21:00:00,58.0,bpm


### Analyze Heart Rate Data within a Fixed Interval Regarding the Extubation Procedure

This section of the code filters the heart rate data to include only the measurements that fall within a specified analysis interval relative to the extubation procedure. The filtered data is then saved to a CSV file for further analysis.

In [10]:
# Create a mask to filter heart rate data within the analysis interval
mask = (pd.to_datetime(heart_rate_data['CHARTTIME']) >= pd.to_datetime(heart_rate_data['START_ANALYSIS'])) & (pd.to_datetime(heart_rate_data['CHARTTIME']) <= pd.to_datetime(heart_rate_data['END_ANALYSIS']))

# Apply the mask to get the heart rate data within the analysis interval
heart_rate_data_in_analysis = heart_rate_data[mask]

# Save the filtered data to a CSV file
heart_rate_data_in_analysis.to_csv('/Users/felipeakiomatsuoka/Desktop/PIBIC/csvs/heart_rate_data_in_analysis.csv', index=False)

### Load and Clean Heart Rate Data for Analysis

This section of the code loads the heart rate data from a CSV file, drops unnecessary columns, selects relevant columns, and renames them for clarity. This prepares the data for further analysis.


In [ ]:
# Load the heart rate data from the CSV file
heart_rate = pd.read_csv('/Users/felipeakiomatsuoka/Desktop/PIBIC/csvs/heart_rate_data_in_analysis.csv')

# Drop the unnecessary 'Unnamed: 0' column
heart_rate = heart_rate.drop(columns=['Unnamed: 0'])

# Select relevant columns for analysis
heart_rate = heart_rate[['ICUSTAY_ID', 'VALUENUM', 'SUBJECT_ID_x', 'CHARTTIME', 'LABEL']]

# Rename columns for clarity
heart_rate = heart_rate.rename(columns={'SUBJECT_ID_x': 'SUBJECT_ID', 'VALUENUM': 'HEART_RATE'})

### Load and Merge Patient Data for Heart Rate Analysis

This section of the code loads patient admission and demographic data, merges it with heart rate data, and calculates the age of patients at the time of admission. The merged data is then filtered to include only relevant columns and heart rate values above a certain threshold that are physiologically possible.

In [ ]:
# Load admissions data from CSV file
admissions_df = pd.read_csv('/Users/felipeakiomatsuoka/Desktop/PIBIC/csvs/ADMISSIONS.csv')

# Load patients data from CSV file
patients_df = pd.read_csv('/Users/felipeakiomatsuoka/Desktop/PIBIC/csvs/PATIENTS.csv')

# Merge admissions and patients data on 'SUBJECT_ID'
merged_df = pd.merge(admissions_df, patients_df, on='SUBJECT_ID')

# Convert 'ADMITTIME' and 'DOB' columns to datetime format
merged_df['ADMITTIME'] = pd.to_datetime(merged_df['ADMITTIME'])
merged_df['DOB'] = pd.to_datetime(merged_df['DOB'])

# Function to calculate age in years at the time of admission
def calculate_age(admit_time, dob):
    admit_time_py = admit_time.to_pydatetime()
    dob_py = dob.to_pydatetime()
    age = (admit_time_py - dob_py).days / 365.25
    return 89 if age > 89 else age  # Cap age at 89

# Apply the age calculation function to each row
merged_df['AGE'] = merged_df.apply(lambda row: calculate_age(row['ADMITTIME'], row['DOB']), axis=1)
merged_df['AGE'] = merged_df['AGE'].astype(int)  # Convert age to integer

# Merge heart rate data with the merged patient data on 'SUBJECT_ID'
heart_rate = pd.merge(heart_rate, merged_df, on='SUBJECT_ID')

# Select relevant columns for analysis
heart_rate = heart_rate[['SUBJECT_ID', 'ICUSTAY_ID', 'AGE', 'GENDER', 'ETHNICITY', 'CHARTTIME', 'HEART_RATE', 'LABEL']]

# Filter out heart rate values less than or equal to 30
heart_rate = heart_rate.loc[heart_rate['HEART_RATE'] > 30]

### Group and Analyze Heart Rate Data by ICU Stay

This section of the code groups heart rate data by ICU stay, calculates various statistics, and merges these statistics with the original data. The final dataset is saved to a CSV file for further analysis.

In [ ]:
# Group heart rate data by 'ICUSTAY_ID' and calculate statistics
grouped = heart_rate.groupby('ICUSTAY_ID')['HEART_RATE']
grouped_stats = grouped.agg(['mean', 'median', 'std', 'var', 'min', 'max']).reset_index()

# Calculate the range of heart rate values for each ICU stay
grouped_stats['range'] = grouped_stats['max'] - grouped_stats['min']

# Select relevant statistics columns
grouped_stats = grouped_stats[['ICUSTAY_ID', 'mean', 'median', 'std', 'min', 'max', 'range']]

# Count the number of heart rate measurements for each ICU stay
counts = heart_rate.groupby('ICUSTAY_ID')['HEART_RATE'].count()

# Merge the heart rate data with the counts
icu_stays_heart_rate = pd.merge(heart_rate, counts, on='ICUSTAY_ID')

# Drop duplicate rows, keeping the first occurrence
icu_stays_heart_rate = icu_stays_heart_rate.drop_duplicates(subset='ICUSTAY_ID', keep='first')

# Rename columns for clarity
icu_stays_heart_rate = icu_stays_heart_rate.rename(columns={'HEART_RATE_x': 'HEART_RATE', 'HEART_RATE_y': 'COUNT'})

# Drop unnecessary columns
icu_stays_heart_rate = icu_stays_heart_rate.drop(columns=['CHARTTIME', 'HEART_RATE'])

# Merge the heart rate data with the grouped statistics
icu_stays_heart_rate = pd.merge(icu_stays_heart_rate, grouped_stats, on='ICUSTAY_ID')

# Rename the 'LABEL' column to 'EXTUBATION_FAILURE'
icu_stays_heart_rate = icu_stays_heart_rate.rename(columns={'LABEL': 'EXTUBATION_FAILURE'})

# Display the first few rows of the final dataset
icu_stays_heart_rate.head()

# Get the length of the 'EXTUBATION_FAILURE' column
len(icu_stays_heart_rate['EXTUBATION_FAILURE'])

# Save the final dataset to a CSV file
icu_stays_heart_rate.to_csv('/Users/felipeakiomatsuoka/Desktop/PIBIC/csvs/final.csv')